In [1]:
from sklearn.ensemble import RandomForestClassifier
#import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
#import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [2]:
def ROC_AUC(y_true, y_score):
	auc = roc_auc_score(y_true, y_score)
	return auc

In [3]:
fingerprint_list = ['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP', 
'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP', 'MHFP6', 'MAP4']
descriptor_list = ['Property', 'Constitution', 'Autocorr', 'Fragment', 'Charge', 
'Estate', 'MOE', 'Connectivity', 'Topology', 'Kappa', 'Path', 'Matrix', 'InfoContent']

In [4]:
def get_Xye_for_one_fingerprint(feature):
    #feature='AtomPairFP'
    csv_file = "Main.csv"
    df = pd.read_csv(csv_file)
    smiles_col = df.columns[0]
    values_col = df.columns[1]

    csv_feat_file = f"fp/Main_{feature}.csv"
    dff = pd.read_csv(csv_feat_file)

    print('values_col = ',values_col)
    #feats_col = df.columns[2:]

    MASK = -1
    #y = df[values_col].astype('int').fillna(MASK).values
    y = df[values_col].astype('int').fillna(MASK).values
    #X = df[feats_col].values
    X = dff.values
    n_feats = len(dff.columns)

    #read external set
    csv_file_e = "Ext.csv"
    df_e = pd.read_csv(csv_file_e)
    smiles_col = df_e.columns[0]
    values_col = df_e.columns[1]
    csv_feat_file_e = f"fp/Ext_{feature}.csv"
    df_ef = pd.read_csv(csv_feat_file_e)

    MASK = -1
    y_e = df_e[values_col].astype('int').fillna(MASK).values
    X_e = df_ef.values
    return X,y,X_e,y_e

In [5]:
def get_Xye_for_one_descriptor(feature):
    #feature='AtomPairFP'
    csv_file = "Main.csv"
    df = pd.read_csv(csv_file)
    smiles_col = df.columns[0]
    values_col = df.columns[1]

    csv_feat_file = f"fp/Main_{feature}S.csv"
    dff = pd.read_csv(csv_feat_file)

    print('values_col = ',values_col)
    #feats_col = df.columns[2:]

    MASK = -1
    #y = df[values_col].astype('int').fillna(MASK).values
    y = df[values_col].astype('int').fillna(MASK).values
    #X = df[feats_col].values
    X = dff.values
    n_feats = len(dff.columns)

    #read external set
    csv_file_e = "Ext.csv"
    df_e = pd.read_csv(csv_file_e)
    smiles_col = df_e.columns[0]
    values_col = df_e.columns[1]
    csv_feat_file_e = f"fp/Ext_{feature}S.csv"
    df_ef = pd.read_csv(csv_feat_file_e)

    MASK = -1
    y_e = df_e[values_col].astype('int').fillna(MASK).values
    X_e = df_ef.values
    return X,y,X_e,y_e

# run fcnn model with param

In [8]:
# seem the keras model can only run once inside a function, otherwise will get GPU errors.
def run_one_keras_fcnn_with_param(feature,i):   
    model = Sequential()
    model.add(Dense(n_dense, input_shape=(n_feats,), activation='relu'),)
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    #print(model.summary)
    pklf = f'./rand_MorganFP/get_split/split_indices_fold{i}.pckl'
    train_idx, valid_idx, test_idx = pd.read_pickle(pklf)
    print(len(train_idx), len(valid_idx), len(test_idx))
    #get X and Y (main set and external set)
    X_T = X[train_idx];y_T = y[train_idx]
    X_valid = X[valid_idx];y_valid = y[valid_idx]
    X_test = X[test_idx];y_test = y[test_idx]     
    y_T_2dim=y_T[:,np.newaxis]  ### change shape from(2769,) to (2769, 1)
    y_valid_2dim = y_valid[:,np.newaxis]
    print(y_T_2dim.shape)
    #model.fit(X_T, y_T_2dim,batch_size=4096,verbose=0, epochs=100)
    ### using early stop by monitor val_loss of X_valid, y_valid_2dim
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_T, y_T_2dim, batch_size=4096, verbose=0, epochs=100, 
              validation_data=(X_valid, y_valid_2dim), callbacks=[early_stopping])    
    ## prediction
    y_prob_T = model.predict(X_T) 
    train_roc_auc = ROC_AUC(y_T,y_prob_T[:, 0])
    print('train_roc_auc = ', train_roc_auc )
    y_prob_valid=model.predict(X_valid)  ## no need other parameters here
    valid_roc_auc = ROC_AUC(y_valid,y_prob_valid[:, 0])
    y_prob_test=model.predict(X_test)  
    test_roc_auc = ROC_AUC(y_test,y_prob_test[:, 0])
    print('test_roc_auc, valid_roc_auc = ', test_roc_auc, valid_roc_auc)
    y_prob_e=model.predict(X_e)  ## no need other parameters here
    e_roc_auc = ROC_AUC(y_e,y_prob_e[:, 0])
    print('e_roc_auc = ', e_roc_auc)    
    ## save predition
    if not os.path.exists('fcnn_new'):os.mkdir('fcnn_new')
    pd.DataFrame(y_prob_T[:,0],columns=['prob']).to_csv(f'fcnn_new/p_{optimizer}_{n_dense}_train_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_valid[:, 0],columns=['prob']).to_csv(f'fcnn_new/p_{optimizer}_{n_dense}_val_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_test[:,0],columns=['prob']).to_csv(f'fcnn_new/p_{optimizer}_{n_dense}_test_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_e[:,0],columns=['prob']).to_csv(f'fcnn_new/p_{optimizer}_{n_dense}_Ext_{feature}_fold{i}.csv',index=False) 
    ## save model
    #model_save_name = f'fcnn_new/model_{feature}_fold{i}.h5'
    #model.save(model_save_name)    
    model = ''
    ## empty gpu memory
    import torch
    torch.cuda.empty_cache()
    return    

In [22]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"
num_folds = 5
#optimizer =  'rmsprop'
#n_dense = 1000
optimizers=['adam', 'rmsprop'];
hidden_layer_sizes=[200,400, 600, 800,1000]
#activations=['relu', 'tanh'])

#for feature in fingerprint_list:
feature = 'AtomPairFP'
csv_feat_file = f"fp/Main_{feature}.csv"
tdf = pd.read_csv(csv_feat_file,nrows=3)
n_feats = len(tdf.columns)
print(f'processing feat: {feature}')
X,y,X_e,y_e = get_Xye_for_one_fingerprint(feature)

for optimizer in optimizers:
    for n_dense in hidden_layer_sizes:    
        for i in range(num_folds): 
            run_one_keras_fcnn_with_param(feature,i)

processing feat: AtomPairFP
values_col =  label
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9890276227358132
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9573039215686274 0.9199627300256231
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9296506137865911
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9925262693329637
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9173284313725489 0.938051677098379
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9166910878838201
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9934145832825932
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9649110149110149 0.967135333863275
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.93863762169906

e_roc_auc =  0.9200449350395623
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9969209704327918
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9673641173641173 0.9687251589825118
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9450196997818372
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9946286572676781
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9154260299625468 0.9234405458089668
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9290319429520366
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9906420009220839
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9463833061741808 0.9339729397293972
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9094298459835239
2769 346 34

2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9924946211771938
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9499139960166576 0.9346617466174661
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9111556119957019
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9942385640281205
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.961078431372549 0.9234567901234568
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9328742144508482
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9869515674236171
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9105637254901962 0.9379446851762693
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.898684510436
2769 346 347
(2769, 1)
87/87 [==================

In [23]:
# coding:utf-8
import argparse
#===============================================================
import os,sys,glob,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from scipy.stats import ttest_ind,linregress;
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import matthews_corrcoef
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
from sklearn.metrics import auc, mean_absolute_error, mean_squared_error, precision_recall_curve, \
r2_score,roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from sklearn.metrics import f1_score,confusion_matrix

In [11]:
h1_list = ['r2_score','pearson_r2','rmse','mse','mae','roc','prc','p-stat','Recall',
'Precision','f1','BA','accuracy', 'TN', 'FP', 'FN','TP','SP','SE','NPV','MCC']
def polyfit(x, y, degree):
	#coeffs = numpy.polyfit(x, y, degree) # Polynomial Coefficients
	correlation = numpy.corrcoef(x, y)[0,1]
	return correlation**2

def calc_class_roc_prc(y_true,y_pred,pos_label=1):
	#print('y_true =', y_true)
	#y_ = [y for y in y_true if y not in [0,1]]
	#print('y_ = ',y_)   ## -9223372036854775808, must be NaN
	x_roc=roc_auc_score(y_true,y_pred)
	if pos_label==0:x_roc = 1 - x_roc		
	x_precision, x_recall, x_thresholds = precision_recall_curve(y_true,y_pred)
	x_prc = auc(x_recall, x_precision)
	return {'roc':x_roc,'prc':x_prc}

def calc_class_other_stats_with_threshold(y_true, y_pred, threshold=0.5):
	ind0=np.where(y_true==0)[0];ind1=np.where(y_true==1)[0]
	y_pred0=y_pred[ind0];y_pred1=y_pred[ind1];
	p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
	#print('p_stat = ', p_0_1)
	y_hard_pred = [1 if p > threshold else 0 for p in y_pred] # binary prediction
	#x_cohen_kappa = cohen_kappa_score(y_true, y_pred, weights='linear')	
	### not understand, can problem, Can't handle mix of binary and continuous
	x_cohen_kappa = cohen_kappa_score(y_true, y_hard_pred, weights='linear')
	## True and false values
	TN, FP, FN, TP = confusion_matrix(y_true, y_hard_pred).ravel()	
	#TP = sum((y_true == 1) & (y_hard_pred == 1))
	#TN = sum((y_true == 0) & (y_hard_pred == 0))
	#FN = sum((y_true == 1) & (y_hard_pred == 0))
	#FP = sum((y_true == 0) & (y_hard_pred == 1))
	# SE (Sensitivity), hit rate, recall, or true positive rate
	x_Recall = TP/(TP+FN)
	SE=x_Recall
	# Precision or positive predictive value
	x_Precision = TP/(TP+FP)
	#f1_score = (2*x_Precision*x_Recall)/ (x_Precision + x_Recall)
	f1 = f1_score(y_true, y_hard_pred)
	# CCR (Correct classification rate), BA (balanced accuracy)
	# Specificity or true negative rate
	SP = TN/(TN+FP) 
	x_BA = (SE + SP)/2
	x_accuracy = accuracy_score(y_true, y_hard_pred)
	# Negative predictive value
	NPV = TN/(TN+FN)
	x_MCC = matthews_corrcoef(y_true, y_hard_pred)
	other_stats = {'p_stat':p_0_1,'Recall':x_Recall,'Precision':x_Precision, 'f1':f1,'BA':x_BA,
	'accuracy':x_accuracy, 'TN':TN, 'FP':FP, 'FN':FN,'TP':TP,'SP':SP,'SE':SE,
	'NPV':NPV,'MCC':x_MCC,'cohen_kappa':x_cohen_kappa}  # 
	return other_stats
def calc_ef_threholds(y_true,y_pred,ef_threholds=[0.01,0.05,0.1]):
	# y_true,y_pred are np.array type
	df = pd.DataFrame()   ## merge  y_true,y_pred as df 's  two cols
	df['y_true'] = y_true;df['y_pred'] = y_pred
	n_actives = len(df[df['y_true']==1].index)
	n_total = len(df.index)
	random_rate = n_actives/ n_total
	df.sort_values(by='y_pred',ascending=False,inplace=True)
	EFs = {}
	for ef_threhold in ef_threholds:
		screen_range = int(np.ceil(n_total * ef_threhold))
		screen_rate = sum(df['y_true'][:screen_range]) / screen_range
		#print('n_actives,n_total = ',n_actives,n_total)
		print('random_rate,screen_range,screen_rate = ', random_rate,screen_range,screen_rate)
		EF = screen_rate / random_rate
		EF_name = 'EF_{}'.format(ef_threhold)
		#print('{} = {}'.format(EF_name,EF))
		EFs.update({EF_name:EF})
	return EFs
def list_stat_for_class(true_file,pred_file,pos_label,ef_threholds):
    t_df = pd.read_csv(true_file)
    p_df = pd.read_csv(pred_file)
    #t_values = t_df[label_col].values
    t_values = t_df.iloc[:,1].values.astype(int)
    if model == 'DMPNN': p_values = p_df.iloc[:,1].values
    else: p_values = p_df.iloc[:,0].values
    ind = np.where((t_values==0) | (t_values==1))[0]	 ### to remove those NaN label
    #print('ind = ',ind)
    y_true = t_values
    [ind]
    y_pred = p_values[ind]
    print('y_true[:10], y_pred[:10] =',y_true[:10], y_pred[:10])
    dic = {}
    dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
    dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
    dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
    w_line= pred_file +',' + ','.join([str(dic[x]) for x in header])+'\n'
    f = open(eva_csv_name, 'a'); f.write(w_line); f.close()
    return 

In [12]:
efs=ef_threholds=[0.01,0.02,0.05]
thre=threshold=0.5
pos_label = 1

In [13]:
fingerprint_list = ['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP', 
'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP', 'MHFP6', 'MAP4']
descriptor_list = ['Property', 'Constitution', 'Autocorr', 'Fragment', 'Charge', 
'Estate', 'MOE', 'Connectivity', 'Topology', 'Kappa', 'Path', 'Matrix', 'InfoContent']

models = ['fcnn_new'] #['xgb','fcnn','log','svm','rf','DMPNN']
num_folds = 5
#true_file = 'hide/rand_check/fold_0/train_full.csv'
b_list = ['train','val','test','Ext']   ## DMPNN use this

true_file_pat = 'rand_/fold_{i}/{b}_full.csv' 
true_file_e = 'Ext.csv'

pred_file_pat = '{model}/p_{b}_{feature}_fold{i}.csv'  # include DMPNN

In [14]:
eva_csv_name = 'eva_with_rmSim_hyper.csv'
dic = {}
y_true,y_pred =np.array([1,0]), np.array([0.9,0.1])
dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
header=list(dic.keys()); #header=sorted(header)
tmp = [x for x in h1_list if x in header] + [x for x in header if x not in h1_list]
header = tmp
print('header=',header)
f = open(eva_csv_name, 'a');f.write('col_names,' + ','.join(header)+'\n');f.close()

random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
header= ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE', 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']


/tmp/ipykernel_2644714/877004199.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [24]:
fps= ['AtomPairFP'] #fingerprint_list+descriptor_list
#for feature in fps:
feature = 'AtomPairFP'
#for model in models:
model = 'fcnn_new'    
for optimizer in optimizers:
    for n_dense in hidden_layer_sizes:
        print(f'processing optimizer: {optimizer}; n_dense: {n_dense}')
        for b in b_list: 
            for i in range(num_folds):
                if b=='Ext': true_file = 'Ext.csv'
                else: true_file = f'rand_{feature}/fold_{i}/{b}_full.csv'             
                pred_file = f'{model}/p_{optimizer}_{n_dense}_{b}_{feature}_fold{i}.csv'  
                # except DMPNN, other model use this
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)  

processing optimizer: adam; n_dense: 200
true_file,pred_file =  rand_AtomPairFP/fold_0/train_full.csv fcnn_new/p_adam_200_train_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.9864174  0.9690017  0.69576424 0.9996107  0.09764577 0.98655325
 0.76818    0.98760843 0.99741936 0.9908444 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/train_full.csv fcnn_new/p_adam_200_train_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.08493373 0.9995633  0.9916385  0.9968451  0.99973565 0.99947697
 0.9882609  0.9626547  0.9992324  0.04497145]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =

random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
true_file,pred_file =  Ext.csv fcnn_new/p_adam_200_Ext_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9874844  0.9970005  0.9982698  0.99920505 0.9999119  0.9995981
 0.9707879  0.967765   0.9940859  0.99388254]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
processing optimizer: adam; n_dense: 400
true_file,pred_file =  rand_AtomPairFP/fold_0/train_full.csv fcnn_new/p_adam_400_train_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.98563415 0.95524216 0.72459763 0.99896836 0.08727585 0.97424376
 0.7932657  0.98402125 0.9984504  0.9858498 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 2

true_file,pred_file =  rand_AtomPairFP/fold_1/train_full.csv fcnn_new/p_adam_600_train_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.06439654 0.9996797  0.9905433  0.99661773 0.9995728  0.9994987
 0.99299157 0.96517557 0.9984107  0.02049768]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/train_full.csv fcnn_new/p_adam_600_train_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.47384882 0.98119146 0.9952809  0.9993063  0.99922085 0.9991792
 0.4777302  0.774624   0.96545005 0.9717557 ]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/train_full.csv fcn

random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/train_full.csv fcnn_new/p_adam_800_train_AtomPairFP_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.19830835 0.9991111  0.76965135 0.99389344 0.9900495  0.15025015
 0.9963917  0.9752019  0.99126655 0.7105958 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_4/train_full.csv fcnn_new/p_adam_800_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9601419  0.9825821  0.978965   0.9960007  0.03148596 0.77954113
 0.99604344 0.04282047 0.31719694 0.04085411]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 

random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/train_full.csv fcnn_new/p_adam_1000_train_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.07235043 0.99946076 0.9915571  0.9961383  0.99928755 0.9990928
 0.9911237  0.9670934  0.998492   0.01810333]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/train_full.csv fcnn_new/p_adam_1000_train_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.46109858 0.985479   0.9950139  0.9996947  0.9990031  0.9992999
 0.4500243  0.71000063 0.966194   0.9744082 ]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 

true_file,pred_file =  rand_AtomPairFP/fold_1/train_full.csv fcnn_new/p_rmsprop_200_train_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.04376838 0.9989938  0.982524   0.99429905 0.99858093 0.99794215
 0.9750122  0.96026695 0.9971908  0.02354004]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/train_full.csv fcnn_new/p_rmsprop_200_train_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.4702915  0.96955293 0.9932162  0.99954575 0.99943167 0.999577
 0.5163883  0.8329239  0.97803533 0.96499443]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/train_full.c

random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/train_full.csv fcnn_new/p_rmsprop_400_train_AtomPairFP_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.12805533 0.97892225 0.6858614  0.97006446 0.96116316 0.0766265
 0.9653004  0.8435171  0.9547931  0.47354856]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_4/train_full.csv fcnn_new/p_rmsprop_400_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9744947  0.98765486 0.98507905 0.99698657 0.0189892  0.8116521
 0.9962159  0.04124927 0.2792147  0.04777914]
random_rate,screen_range,screen_rate =  0.7833152762730228

true_file,pred_file =  rand_AtomPairFP/fold_4/train_full.csv fcnn_new/p_rmsprop_600_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9601583  0.97660965 0.98622316 0.9960407  0.02183216 0.7835623
 0.9961469  0.04783063 0.28571108 0.0501558 ]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_0/val_full.csv fcnn_new/p_rmsprop_600_val_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.99190384 0.2366188  0.8407425  0.81432766 0.9988852  0.868989
 0.99742186 0.9690869  0.7525914  0.99925214]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/val_full.csv fcnn_ne

true_file,pred_file =  rand_AtomPairFP/fold_1/val_full.csv fcnn_new/p_rmsprop_800_val_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [0.985449   0.00254545 0.9718112  0.9993962  0.9999999  0.436577
 0.0323498  0.99972886 0.99995613 0.7499975 ]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/val_full.csv fcnn_new/p_rmsprop_800_val_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 0 1 1 1 1 0 1] [0.97907674 0.99539113 0.10638509 0.0409858  0.9603713  0.99938786
 0.99301857 0.9913133  0.31652853 0.9767495 ]
random_rate,screen_range,screen_rate =  0.7861271676300579 4 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 7 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/val_full.csv fcnn_new/p_rm

true_file,pred_file =  rand_AtomPairFP/fold_4/val_full.csv fcnn_new/p_rmsprop_1000_val_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 0 1 0 1 1 0 1 0 1] [0.99998474 0.05661208 0.9983353  0.6669846  0.9999676  0.9835625
 0.63361794 0.96374166 0.91695464 0.8627944 ]
random_rate,screen_range,screen_rate =  0.7832369942196532 4 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 7 1.0
random_rate,screen_range,screen_rate =  0.7832369942196532 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_0/test_full.csv fcnn_new/p_rmsprop_1000_test_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [0 0 1 1 0 1 1 1 1 1] [0.00100355 0.02962431 0.9911506  0.9377057  0.0234102  0.9990446
 0.98479176 0.9943006  0.27993503 0.95953995]
random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/test_full.csv fcnn_new

In [25]:
print('eva_csv_name = ', eva_csv_name)
df = pd.read_csv(eva_csv_name)

eva_csv_name =  eva_with_rmSim_hyper.csv


In [26]:
df.head(5)

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,TP,SP,SE,NPV,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa
0,fcnn_new/p_adam_200_train_AtomPairFP_fold0.csv,0.990334,0.997194,0.987996,0.965268,0.976500,0.930151,0.962802,526,77,...,2140,0.872305,0.987996,0.952899,0.888763,1.278393,1.278393,1.278393,0.0,0.887380
1,fcnn_new/p_adam_200_train_AtomPairFP_fold1.csv,0.992074,0.997656,0.984201,0.966682,0.975363,0.932943,0.961358,544,73,...,2118,0.881686,0.984201,0.941176,0.886624,1.286710,1.286710,1.286710,0.0,0.885856
2,fcnn_new/p_adam_200_train_AtomPairFP_fold2.csv,0.994502,0.998429,0.987969,0.971780,0.979807,0.942997,0.968220,546,62,...,2135,0.898026,0.987969,0.954545,0.905936,1.281351,1.281351,1.281351,0.0,0.905255
3,fcnn_new/p_adam_200_train_AtomPairFP_fold3.csv,0.992597,0.997861,0.987073,0.966983,0.976925,0.933006,0.963525,530,73,...,2138,0.878939,0.987073,0.949821,0.891046,1.278393,1.278393,1.278393,0.0,0.889975
4,fcnn_new/p_adam_200_train_AtomPairFP_fold4.csv,0.991923,0.997705,0.987552,0.967480,0.977413,0.933776,0.964247,528,72,...,2142,0.880000,0.987552,0.951351,0.892823,1.276625,1.276625,1.276625,0.0,0.891742


In [27]:
import re
# p_adam_800_train_AtomPairFP_fold1.csv
pat = re.compile("p_(\w+)_(\d+)_(\w+)_.*_fold(\d+)")
col_names = df['col_names'].values
op_list = [];n_dense_list = [];data_list = [];fold_list = []
for col in col_names:
    #m,tmp = col.split('/')    
    mat = re.findall(pat,col)
    op,n_dense,dt,fold = mat[0]
    op_list.append(op);n_dense_list.append(n_dense);data_list.append(dt);fold_list.append(fold)
df['optimizer'] = op_list
df['n_dense'] = n_dense_list
df['data_type'] = data_list
df['fold'] = fold_list

In [28]:
test_df=df[df['data_type']=='test']
ext_df=df[df['data_type']=='Ext']
train_df=df[df['data_type']=='train']
val_df=df[df['data_type']=='train']

In [29]:
def group_factor12_for_one_performance_sort(factor1, factor2, xeva = 'roc'):
    group_test = test_df.groupby([factor1, factor2]).mean()[xeva]
    group_test_std = test_df.groupby([factor1, factor2]).std()[xeva]
    group_ext = ext_df.groupby([factor1, factor2]).mean()[xeva]
    group_ext_std = ext_df.groupby([factor1, factor2]).std()[xeva]
    group_train = train_df.groupby([factor1, factor2]).mean()[xeva]
    group_train_std = train_df.groupby([factor1, factor2]).std()[xeva]
    group_val = val_df.groupby([factor1, factor2]).mean()[xeva]
    group_val_std = val_df.groupby([factor1, factor2]).std()[xeva]
    """
    dfl = [pd.DataFrame() for i in [6,7,8,9]] 
    groupl = [group_val for i in [6,7,8,9]] 
    group_stdl = [group_val_std for i in [6,7,8,9]] 
    tmp_list = []
    for i,sim in enumerate([6,7,8,9]):
        #dfl[0]=df[df['data']=='train']
        dfl[i]=df[df['data']==f'ExtRmSim0{sim}']
        groupl[i] = dfl[i].groupby([factor1, factor2]).mean()[xeva]
        group_stdl[i] = dfl[i].groupby([factor1, factor2]).std()[xeva]
        tmp_list.append(groupl[i]);tmp_list.append(group_stdl[i])
    """
    grouped_factor12 = pd.concat([group_train,group_train_std,group_val,group_val_std,
                group_test,group_test_std,group_ext,group_ext_std],axis=1)  # +tmp_list
    grouped_factor12.columns = [f'train_{xeva}',f'train_{xeva}_std',f'val_{xeva}',f'val_{xeva}_std',
              f'test_{xeva}',f'test_{xeva}_std', f'ext_{xeva}',f'ext_{xeva}_std'] #+ \
            #['rmSim_0.6','rmSim_0.6_std','rmSim_0.7','rmSim_0.7_std',
             #'rmSim_0.8','rmSim_0.8_std', 'rmSim_0.9','rmSim_0.9_std']
    grouped_factor12.to_csv(f'grouped_fcnn_{xeva}.csv')
    grouped_factor12 = pd.read_csv(f'grouped_fcnn_{xeva}.csv')
    grouped_sort=grouped_factor12.sort_values(by=[f'ext_{xeva}',f'test_{xeva}',],ascending=False)
    grouped_sort.to_csv(f'grouped_fcnn_sort_{xeva}.csv',index=False)
    return grouped_sort

In [30]:
factor1, factor2 = 'optimizer','n_dense'
group_factor12_for_one_performance_sort(factor1, factor2, xeva = 'roc')
group_factor12_for_one_performance_sort(factor1, factor2, xeva = 'accuracy')

,optimizer,n_dense,train_accuracy,train_accuracy_std,val_accuracy,val_accuracy_std,test_accuracy,test_accuracy_std,ext_accuracy,ext_accuracy_std
3,adam,600,0.965692,0.007425,0.965692,0.007425,0.908357,0.013087,0.893636,0.013025
4,adam,800,0.967136,0.006255,0.967136,0.006255,0.907781,0.011285,0.893409,0.011209
1,adam,200,0.962839,0.002732,0.962839,0.002732,0.906340,0.011305,0.892955,0.009669
0,adam,1000,0.967714,0.005851,0.967714,0.005851,0.909798,0.012675,0.889545,0.014302
6,rmsprop,200,0.963380,0.002687,0.963380,0.002687,0.903458,0.013466,0.885000,0.011892
2,adam,400,0.963091,0.006399,0.963091,0.006399,0.904323,0.011020,0.884545,0.012531
7,rmsprop,400,0.955869,0.022237,0.955869,0.022237,0.899424,0.021966,0.870455,0.034980
9,rmsprop,800,0.956374,0.016601,0.956374,0.016601,0.896254,0.026482,0.870455,0.029341
8,rmsprop,600,0.951571,0.020049,0.951571,0.020049,0.893948,0.015655,0.868409,0.025901
5,rmsprop,1000,0.938317,0.037893,0.938317,0.037893,0.885879,0.031044,0.850682,0.036349


In [43]:
# seem the keras model can only run once inside a function, otherwise will get GPU errors.
def run_one_keras_fcnn_multi_layers_with_param(hidden_layer_sizes, feature,i):   
    model = Sequential()
    #model.add(Dense(n_dense, input_shape=(n_feats,), activation='relu'),)
    
    if type(hidden_layer_sizes)==int:hidden_layer_sizes=[hidden_layer_sizes]
    for j, size in enumerate(hidden_layer_sizes):
        if j == 0:
            model.add(Dense(size, input_shape=(X.shape[1],), activation='relu'))
        else:
            model.add(Dense(size, activation='relu'))     
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])    
    
    #print(model.summary)
    pklf = f'./rand_MorganFP/get_split/split_indices_fold{i}.pckl'
    train_idx, valid_idx, test_idx = pd.read_pickle(pklf)
    print(len(train_idx), len(valid_idx), len(test_idx))
    #get X and Y (main set and external set)
    X_T = X[train_idx];y_T = y[train_idx]
    X_valid = X[valid_idx];y_valid = y[valid_idx]
    X_test = X[test_idx];y_test = y[test_idx]     
    y_T_2dim=y_T[:,np.newaxis]  ### change shape from(2769,) to (2769, 1)
    y_valid_2dim = y_valid[:,np.newaxis]
    print(y_T_2dim.shape)
    #model.fit(X_T, y_T_2dim,batch_size=4096,verbose=0, epochs=100)
    ### using early stop by monitor val_loss of X_valid, y_valid_2dim
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_T, y_T_2dim, batch_size=4096, verbose=0, epochs=100, 
              validation_data=(X_valid, y_valid_2dim), callbacks=[early_stopping])    
    ## prediction
    y_prob_T = model.predict(X_T) 
    train_roc_auc = ROC_AUC(y_T,y_prob_T[:, 0])
    print('train_roc_auc = ', train_roc_auc )
    y_prob_valid=model.predict(X_valid)  ## no need other parameters here
    valid_roc_auc = ROC_AUC(y_valid,y_prob_valid[:, 0])
    y_prob_test=model.predict(X_test)  
    test_roc_auc = ROC_AUC(y_test,y_prob_test[:, 0])
    print('test_roc_auc, valid_roc_auc = ', test_roc_auc, valid_roc_auc)
    y_prob_e=model.predict(X_e)  ## no need other parameters here
    e_roc_auc = ROC_AUC(y_e,y_prob_e[:, 0])
    print('e_roc_auc = ', e_roc_auc)    
    ## save predition
    n1,n2 = hidden_layer_sizes
    if not os.path.exists('fcnn_2layer'):os.mkdir('fcnn_2layer')
    pd.DataFrame(y_prob_T[:,0],columns=['prob']).to_csv(f'fcnn_2layer/p_{n1}_{n2}_train_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_valid[:, 0],columns=['prob']).to_csv(f'fcnn_2layer/p_{n1}_{n2}_val_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_test[:,0],columns=['prob']).to_csv(f'fcnn_2layer/p_{n1}_{n2}_test_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_e[:,0],columns=['prob']).to_csv(f'fcnn_2layer/p_{n1}_{n2}_Ext_{feature}_fold{i}.csv',index=False) 
    ## save model
    #model_save_name = f'fcnn_new/model_{feature}_fold{i}.h5'
    #model.save(model_save_name)    
    model = ''
    ## empty gpu memory
    import torch
    torch.cuda.empty_cache()
    return    

In [44]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"
num_folds = 5
#optimizer =  'rmsprop'
#n_dense = 1000
optimizers=['adam', 'rmsprop'];
#hidden_layer_sizes=[200,400, 600, 800,1000]
#activations=['relu', 'tanh'])

#for feature in fingerprint_list:
feature = 'AtomPairFP'
csv_feat_file = f"fp/Main_{feature}.csv"
tdf = pd.read_csv(csv_feat_file,nrows=3)
n_feats = len(tdf.columns)
print(f'processing feat: {feature}')
X,y,X_e,y_e = get_Xye_for_one_fingerprint(feature)
optimizer = 'adam'
#for optimizer in optimizers:
hidden_layer_sizes_list = [[30,30],[40,40],[50,50],[50,100],[50,200],[50,400],[100,100],[100,200],]
for hidden_layer_sizes in hidden_layer_sizes_list:    
    for i in range(num_folds): 
        #run_one_keras_fcnn_with_param(feature,i)
        run_one_keras_fcnn_multi_layers_with_param(hidden_layer_sizes, feature,i)

processing feat: AtomPairFP
values_col =  label
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 973us/step
train_roc_auc =  0.9822831058618879
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9509313725490196 0.9186116934544608
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9111230503728306
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9770945424858261
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9057598039215686 0.9365537901888407
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9021686040832274
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 938us/step
train_roc_auc =  0.9814790149540905
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9585137085137085 0.9657442368839427
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.92063104

e_roc_auc =  0.9105369411611475
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.99390701490538
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9661616161616161 0.9599811208267091
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.937660773012927
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9926104319890239
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9024578651685393 0.9144736842105263
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9201751815310476
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9798367143076687
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9369228680065183 0.9262976629766299
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.9027221516720394
2769 346 347
(

In [39]:
efs=ef_threholds=[0.01,0.02,0.05]
thre=threshold=0.5
pos_label = 1

models = ['fcnn_2layer'] #['xgb','fcnn','log','svm','rf','DMPNN']
num_folds = 5
#true_file = 'hide/rand_check/fold_0/train_full.csv'
b_list = ['train','val','test','Ext']   ## DMPNN use this

true_file_pat = 'rand_/fold_{i}/{b}_full.csv' 
true_file_e = 'Ext.csv'

pred_file_pat = '{model}/p_{b}_{feature}_fold{i}.csv'  # include DMPNN

In [45]:
eva_csv_name = 'eva_with_rmSim_hyper_2layer.csv'
dic = {}
y_true,y_pred =np.array([1,0]), np.array([0.9,0.1])
dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
header=list(dic.keys()); #header=sorted(header)
tmp = [x for x in h1_list if x in header] + [x for x in header if x not in h1_list]
header = tmp
print('header=',header)
f = open(eva_csv_name, 'a');f.write('col_names,' + ','.join(header)+'\n');f.close()

fps= ['AtomPairFP'] #fingerprint_list+descriptor_list
#for feature in fps:
feature = 'AtomPairFP'
#for model in models:
model = 'fcnn_2layer'    
for optimizer in optimizers:
    for hidden_layer_sizes in hidden_layer_sizes_list:
        n1,n2=hidden_layer_sizes
        print(f'processing hidden_layer_sizes: {n1}:{n2}')
        for b in b_list: 
            for i in range(num_folds):
                if b=='Ext': true_file = 'Ext.csv'
                else: true_file = f'rand_{feature}/fold_{i}/{b}_full.csv'             
                pred_file = f'{model}/p_{n1}_{n2}_{b}_{feature}_fold{i}.csv'  
                # except DMPNN, other model use this
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)  

random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
header= ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE', 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']
processing hidden_layer_sizes: 30:30
true_file,pred_file =  rand_AtomPairFP/fold_0/train_full.csv fcnn_2layer/p_30_30_train_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.9838244  0.9441632  0.6726596  0.99958867 0.09200896 0.99103093
 0.7412985  0.9907603  0.9954117  0.9853839 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/train_full.csv fcnn_2layer/p_30_30_train_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.441742

/tmp/ipykernel_2644714/877004199.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/train_full.csv fcnn_2layer/p_40_40_train_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.41538003 0.97683233 0.9892062  0.99953353 0.99928063 0.9984414
 0.7908873  0.7423655  0.94938356 0.9467458 ]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/train_full.csv fcnn_2layer/p_40_40_train_AtomPairFP_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.23279981 0.9995864  0.7348623  0.9952755  0.99586666 0.17500874
 0.9972818  0.9930408  0.99000883 0.771967  ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1

y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.25822774 0.9995634  0.846966   0.9940421  0.9925632  0.20456459
 0.99817824 0.98619634 0.9931763  0.7201368 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_4/train_full.csv fcnn_2layer/p_50_50_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9788326  0.9956584  0.984059   0.9987489  0.032673   0.77529424
 0.9985843  0.03686917 0.23054636 0.06048291]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_0/val_full.csv fcnn_2layer/p_50_50_val_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.9995214  0.272

true_file,pred_file =  rand_AtomPairFP/fold_0/val_full.csv fcnn_2layer/p_50_100_val_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.9995975  0.3462386  0.98784757 0.9604749  0.9999255  0.9818201
 0.99735826 0.99643886 0.91448694 0.9990798 ]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/val_full.csv fcnn_2layer/p_50_100_val_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [0.92738825 0.00825817 0.91496974 0.99501884 0.99998844 0.14268805
 0.005589   0.9986304  0.9968658  0.28899923]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/val_full.csv fcnn_2layer/p_50

...

true_file,pred_file =  rand_AtomPairFP/fold_4/train_full.csv fcnn_2layer/p_50_50_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9788326  0.9956584  0.984059   0.9987489  0.032673   0.77529424
 0.9985843  0.03686917 0.23054636 0.06048291]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_AtomPairFP/fold_0/val_full.csv fcnn_2layer/p_50_50_val_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.9995214  0.2729283  0.9799059  0.9238681  0.99995494 0.9723565
 0.99882585 0.99768066 0.78764474 0.99918395]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/val_full.csv fcnn_2layer

random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/val_full.csv fcnn_2layer/p_50_100_val_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 0 1 1 1 1 0 1] [0.98025435 0.9983437  0.281273   0.09085096 0.9822468  0.99967086
 0.9977246  0.99828917 0.25555933 0.99699605]
random_rate,screen_range,screen_rate =  0.7861271676300579 4 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 7 1.0
random_rate,screen_range,screen_rate =  0.7861271676300579 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_3/val_full.csv fcnn_2layer/p_50_100_val_AtomPairFP_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.996358   0.83221674 0.9898012  0.9051867  0.9671473  0.662873
 0.9999677  0.99983    0.9652762  0.7636797 ]
random_rate,screen_range,screen_rate =  0.7803468208092486 4 1.0
random_rate

random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_1/test_full.csv fcnn_2layer/p_50_200_test_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 0 0 0 0 1 0 0] [0.99593014 0.77458245 0.7025937  0.91336936 0.08535898 0.13774325
 0.01043731 0.9387479  0.05568969 0.97055143]
random_rate,screen_range,screen_rate =  0.7838616714697406 4 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 7 1.0
random_rate,screen_range,screen_rate =  0.7838616714697406 18 1.0
true_file,pred_file =  rand_AtomPairFP/fold_2/test_full.csv fcnn_2layer/p_50_200_test_AtomPairFP_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 0 1 1 1 1 1] [0.06740459 0.99841094 0.9772958  0.2795832  0.236058   0.91489327
 0.9964001  0.99248886 0.98764634 0.97113174]
random_rate,screen_range,screen_rate =  0.7780979827089337 4 1.0
rando

random_rate,screen_range,screen_rate =  0.7579250720461095 4 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 7 1.0
random_rate,screen_range,screen_rate =  0.7579250720461095 18 1.0
true_file,pred_file =  Ext.csv fcnn_2layer/p_50_400_Ext_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9846096  0.99886644 0.9944614  0.99982053 0.9999777  0.99991894
 0.93346536 0.9492076  0.99219817 0.99242365]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
true_file,pred_file =  Ext.csv fcnn_2layer/p_50_400_Ext_AtomPairFP_fold1.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.9401712  0.9997663  0.9993537  0.9999213  0.99997103 0.9999515
 0.98703605 0.98508227 0.997806   0.9957802 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
r

true_file,pred_file =  Ext.csv fcnn_2layer/p_100_100_Ext_AtomPairFP_fold3.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99473155 0.999985   0.9997706  0.99998045 0.99999917 0.99999774
 0.9924615  0.8441482  0.9987515  0.99990165]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
true_file,pred_file =  Ext.csv fcnn_2layer/p_100_100_Ext_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 1 1 1 1 1 1] [0.99218386 0.99935895 0.9982462  0.9997235  0.99998236 0.99992
 0.9851044  0.9390353  0.9974667  0.9962298 ]
random_rate,screen_range,screen_rate =  0.8022727272727272 5 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 9 1.0
random_rate,screen_range,screen_rate =  0.8022727272727272 22 1.0
processing hidden_layer_sizes: 100:200
true_file,pred_file =  rand_AtomPairFP/fold_0/train_full.csv fcnn_2layer/p_100_200_train_Ato

In [63]:
print('eva_csv_name = ', eva_csv_name)
df = pd.read_csv(eva_csv_name)

eva_csv_name =  eva_with_rmSim_hyper_2layer.csv


In [64]:
import re
# fcnn_2layer/p_30_30_train_AtomPairFP_fold0.csv
pat = re.compile("p_(\d+)_(\d+)_(\w+)_.*_fold(\d+)")
col_names = df['col_names'].values
n1_list = [];n2_list = [];data_list = [];fold_list = []
for col in col_names:
    #m,tmp = col.split('/')    
    mat = re.findall(pat,col)
    n1,n2,dt,fold = mat[0]
    n1_list.append(n1);n2_list.append(n2);data_list.append(dt);fold_list.append(fold)
df['n1'] = n1_list
df['n2'] = n2_list
df['data_type'] = data_list
df['fold'] = fold_list

In [65]:
df.head(5)

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa,n1,n2,data_type,fold
0,fcnn_2layer/p_30_30_train_AtomPairFP_fold0.csv,0.982283,0.994670,0.981533,0.958089,0.969669,0.913652,0.951968,510,93,...,0.855841,1.278393,1.278393,1.278393,0.0,0.854399,30,30,train,0
1,fcnn_2layer/p_30_30_train_AtomPairFP_fold1.csv,0.977095,0.992156,0.981877,0.953950,0.967712,0.908281,0.949079,515,102,...,0.849397,1.286710,1.286710,1.286710,0.0,0.847421,30,30,train,1
2,fcnn_2layer/p_30_30_train_AtomPairFP_fold2.csv,0.981479,0.994439,0.979639,0.952745,0.966005,0.903471,0.946190,503,105,...,0.838988,1.281351,1.281351,1.281351,0.0,0.837120,30,30,train,2
3,fcnn_2layer/p_30_30_train_AtomPairFP_fold3.csv,0.991235,0.997461,0.985226,0.970000,0.977554,0.937887,0.964608,537,66,...,0.894566,1.278393,1.278393,1.278393,0.0,0.893960,30,30,train,3
4,fcnn_2layer/p_30_30_train_AtomPairFP_fold4.csv,0.986077,0.995875,0.984786,0.957418,0.970909,0.913226,0.953774,505,95,...,0.860562,1.276625,1.276625,1.276625,0.0,0.858540,30,30,train,4


In [66]:
df.tail(5)

,col_names,roc,prc,Recall,Precision,f1,BA,accuracy,TN,FP,...,MCC,EF_0.01,EF_0.02,EF_0.05,p_stat,cohen_kappa,n1,n2,data_type,fold
315,fcnn_2layer/p_100_200_Ext_AtomPairFP_fold0.csv,0.930041,0.980952,0.963173,0.906667,0.934066,0.780437,0.890909,52,35,...,0.629564,1.246459,1.246459,1.246459,1.227942e-69,0.619939,100,200,Ext,0
316,fcnn_2layer/p_100_200_Ext_AtomPairFP_fold1.csv,0.907476,0.974050,0.932011,0.908840,0.920280,0.776350,0.870455,54,33,...,0.576415,1.246459,1.246459,1.246459,1.523172e-55,0.575117,100,200,Ext,1
317,fcnn_2layer/p_100_200_Ext_AtomPairFP_fold2.csv,0.930237,0.980837,0.949008,0.917808,0.933148,0.802090,0.890909,57,30,...,0.639937,1.246459,1.246459,1.246459,4.378283e-73,0.637300,100,200,Ext,2
318,fcnn_2layer/p_100_200_Ext_AtomPairFP_fold3.csv,0.918547,0.977143,0.974504,0.895833,0.933514,0.757367,0.888636,47,40,...,0.615135,1.246459,1.246459,1.246459,7.957664e-63,0.594554,100,200,Ext,3
319,fcnn_2layer/p_100_200_Ext_AtomPairFP_fold4.csv,0.900931,0.969662,0.917847,0.907563,0.912676,0.769268,0.859091,54,33,...,0.548263,1.246459,1.246459,1.246459,7.267129e-53,0.548030,100,200,Ext,4


In [67]:
test_df=df[df['data_type']=='test']
ext_df=df[df['data_type']=='Ext']
train_df=df[df['data_type']=='train']
val_df=df[df['data_type']=='train']

In [68]:
factor1, factor2 = 'n1','n2'
group_factor12_for_one_performance_sort(factor1, factor2, xeva = 'roc')
group_factor12_for_one_performance_sort(factor1, factor2, xeva = 'accuracy')

,n1,n2,train_accuracy,train_accuracy_std,val_accuracy,val_accuracy_std,test_accuracy,test_accuracy_std,ext_accuracy,ext_accuracy_std
0,100,100,0.964825,0.006574,0.964825,0.006574,0.907205,0.012657,0.893636,0.012253
4,50,100,0.960202,0.004442,0.960202,0.004442,0.904323,0.011103,0.889545,0.016291
3,40,40,0.955507,0.008355,0.955507,0.008355,0.903746,0.015079,0.886364,0.005868
2,30,30,0.953124,0.006635,0.953124,0.006635,0.903746,0.013531,0.885000,0.009040
5,50,200,0.958685,0.008273,0.958685,0.008273,0.903746,0.013934,0.885000,0.008912
7,50,50,0.959408,0.009039,0.959408,0.009039,0.902594,0.015773,0.882727,0.013770
6,50,400,0.957746,0.008043,0.957746,0.008043,0.902017,0.011527,0.882727,0.009535
1,100,200,0.962947,0.007485,0.962947,0.007485,0.902594,0.014682,0.880000,0.013670


In [56]:
group_test = test_df.groupby([factor1, factor2]).mean()['roc']

In [57]:
group_test

n1   n2 
100  200    0.936057
30   200    0.932408
40   200    0.932792
50   200    0.932214
Name: roc, dtype: float64

In [58]:
print(factor1, factor2)

n1 n2


# not used; the grid search would be difficult to use same split

In [35]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def build_fcnn_model_with_layers_roc(optimizer, hidden_layer_sizes, activation):
    model = Sequential()
    if type(hidden_layer_sizes)==int:hidden_layer_sizes=[hidden_layer_sizes]
    for i, size in enumerate(hidden_layer_sizes):
        if i == 0:
            model.add(Dense(size, input_shape=(X.shape[1],), activation=activation))
        else:
            model.add(Dense(size, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=ROC_AUC2)  # metrics=['accuracy']
    return model

In [ ]:
# Define the grid search parameters
param_grid = dict(optimizer=['adam', 'rmsprop'], 
                  hidden_layer_sizes=[400, 600, 800,1000], 
                  activation=['relu', 'tanh'])
# Build the model with KerasClassifier wrapper
model = KerasClassifier(build_fcnn_model_with_layers_roc)

# Conduct the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X, y)
# Report the best parameters and the corresponding score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/tmp/ipykernel_1563813/3944467749.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fcnn_model_with_layers_roc)


> /tmp/__autograph_generated_filerb2nikz9.py(13)tf__ROC_AUC2()
     11                 ag__.ld(pdb).set_trace()
     12 
---> 13                 def get_state():
     14                     return (y_score,)
     15 

ipdb> y_score
<tf.Tensor 'sequential_502/dense_841/Sigmoid:0' shape=(None, 1) dtype=float32>
ipdb> y_score.shape
TensorShape([None, 1])
--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/__autograph_generated_filerb2nikz9.py(13)tf__ROC_AUC2()
     11                 ag__.ld(pdb).set_trace()
     12 
---> 13                 def get_state():
     14                     return (y_score,)
     15 



free GPU memory

In [48]:
# free GPU memory
from numba import cuda 
device = cuda.get_current_device()
device.reset()
cuda.close()

LogisticRegression